In [1]:
# General Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [3]:
# Load in dataset to pandas DataFrame
dataset = pd.read_csv("https://github.com/SBodapati11/Tree-Classification/blob/main/Bank%20Customer%20Churn%20Prediction.csv?raw=true")

In [4]:
# Check for any missing values
sum(dataset.isna().sum())

0

In [5]:
# Check for any null values
sum(dataset.isnull().sum())

0

In [6]:
# Find the statistics of the dataset
dataset.describe()

,customer_id,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [7]:
dataset.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [12]:
# Variables and their meaning
# customer_id - an identification number for the customer (not used)
# credit_score - the customer's credit score
# country - the location the customer's account originated from (France, Germany, or Spain)
# gender - the customer's gender (Male or Female)
# age - the customer's age
# tenure - how long the customer's account has been active for
# balance - the current balance of the customer's account
# products_number - number of products the customer has bought from the bank
# credit_card - whether the customer owns a credit card from the bank (1=yes, 0=no)
# active_member - whether the customer is an active member (1=yes, 0=no)
# estimated_salary - the estimated salary of the customer
# churn - whether the customer has left the bank during some period (1=yes, 0=no) (This is what we are trying to predict)